#### 1. Import pandas library

In [2]:
import pandas as pd

#### 2. Import users table:

In [3]:
users_df=pd.read_csv('users_table.csv')

#### 3. Rename Id column to userId

In [4]:
users_df.rename(columns={'Id':'userId'}, inplace=True)

#### 4. Import posts table:

In [5]:
posts_df=pd.read_csv('posts_table.csv')

#### 5. Rename Id column to postId and OwnerUserId to userId

In [6]:
posts_df.rename(columns={'Id':'postId', "OwnerUserId" : "userId"}, inplace=True)

In [7]:
users_df.columns

Index(['userId', 'Reputation', 'CreationDate', 'DisplayName', 'LastAccessDate',
       'WebsiteUrl', 'Location', 'AboutMe', 'Views', 'UpVotes', 'DownVotes',
       'AccountId', 'Age', 'ProfileImageUrl'],
      dtype='object')

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [8]:
users=users_df[["userId", "Reputation","Views","UpVotes","DownVotes"]]
posts=posts_df[["postId", "Score","userId","ViewCount","CommentCount"]]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [10]:
users_posts=users.merge(posts, on = "userId")
users_posts

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0
...,...,...,...,...,...,...,...,...,...
38957,45934,11,1,0,0,34003,1,115.0,2
38958,46192,36,1,0,0,40667,5,326.0,2
38959,46522,235,13,27,1,17461,3,166.0,0
38960,52371,221,2,0,0,27237,24,3357.0,5


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [93]:
users_posts.isnull().sum()

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [12]:
#I decided to fill them with 0 to translate that they have posts thouhg their count is lacking, and given that at least 10000 rows have values
import numpy as np
users_posts = users_posts.replace(to_replace = np.nan, value = 0) 
users_posts

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0.0,0
1,-1,1,0,5007,1920,8576,0,0.0,0
2,-1,1,0,5007,1920,8578,0,0.0,0
3,-1,1,0,5007,1920,8981,0,0.0,0
4,-1,1,0,5007,1920,8982,0,0.0,0
...,...,...,...,...,...,...,...,...,...
38957,45934,11,1,0,0,34003,1,115.0,2
38958,46192,36,1,0,0,40667,5,326.0,2
38959,46522,235,13,27,1,17461,3,166.0,0
38960,52371,221,2,0,0,27237,24,3357.0,5


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [15]:
#If I understood the question correctly by adjusting you meant cleaning the values that could bring problems in future computations
users_posts_modified = users_posts.dropna()

In [16]:
users_posts_modified

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0.0,0
1,-1,1,0,5007,1920,8576,0,0.0,0
2,-1,1,0,5007,1920,8578,0,0.0,0
3,-1,1,0,5007,1920,8981,0,0.0,0
4,-1,1,0,5007,1920,8982,0,0.0,0
...,...,...,...,...,...,...,...,...,...
38957,45934,11,1,0,0,34003,1,115.0,2
38958,46192,36,1,0,0,40667,5,326.0,2
38959,46522,235,13,27,1,17461,3,166.0,0
38960,52371,221,2,0,0,27237,24,3357.0,5


In [17]:
#confirming all rows with na where eliminated
users_posts_modified.isnull().sum()

userId          0
Reputation      0
Views           0
UpVotes         0
DownVotes       0
postId          0
Score           0
ViewCount       0
CommentCount    0
dtype: int64